- 단계구분도: 경기도내 인구소멸비율

In [1]:
import numpy as np
import json, folium
import pandas as pd

- 경기도 인구소멸비율 구하기

In [2]:
# 파일 읽기
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop.head(5)

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2023,3,도,경기도,13603546,1027575,1325240,1720603,1876012,2282017,...,646221,814119,900263,1120239,1138056,892007,441670,256860,45402,1453
1,2023,3,시군,경기도 가평군,61969,2847,4399,5802,5364,7537,...,2143,2422,2429,3520,5014,6498,3654,2459,402,13
2,2023,3,읍면동,경기도 가평군 가평읍,19548,993,1678,2071,1787,2615,...,784,910,853,1250,1583,1810,1045,759,122,6
3,2023,3,읍면동,경기도 가평군 북면,3834,111,183,229,223,370,...,98,84,103,157,276,517,291,223,39,1
4,2023,3,읍면동,경기도 가평군 상면,5598,135,229,455,377,542,...,94,143,156,247,502,736,395,247,33,0


In [3]:
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(3)
pop = pop[['행정구역구분명','행정구역명', '총 인구수','인구소멸비율']]

In [4]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,61969,0.056
8,시군,경기도 고양시,1077909,0.105
9,구,경기도 고양시 덕양구,495181,0.107
31,구,경기도 고양시 일산동구,293752,0.108
44,구,경기도 고양시 일산서구,288976,0.100


In [5]:
# 수원시, 용인시 등 구가 있는 시는 제거
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)

- geo data 읽어오기

In [6]:
file_name = 'data/경기도_geo_simple.json'
geo_data = json.load(open(file_name, encoding='utf-8'))

- 인구 데이터와 geo 데이터와 인덱스 일치 시키기

In [10]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop, pop.to_csv('data/경기도인구소멸비율.csv', index=False)

(        행정구역구분명         행정구역명   총 인구수  인구소멸비율      시군구
 시군구                                                   
 가평군          시군       경기도 가평군   61969   0.056      가평군
 고양시덕양구        구   경기도 고양시 덕양구  495181   0.107   고양시덕양구
 고양시일산동구       구  경기도 고양시 일산동구  293752   0.108  고양시일산동구
 고양시일산서구       구  경기도 고양시 일산서구  288976   0.100  고양시일산서구
 과천시          시군       경기도 과천시   78654   0.110      과천시
 ...         ...           ...     ...     ...      ...
 포천시          시군       경기도 포천시  158413   0.118      포천시
 하남시          시군       경기도 하남시  137291   0.140      하남시
 하남시          시군       경기도 하남시  137291   0.140      하남시
 화성시          시군       경기도 화성시  378418   0.152      화성시
 화성시          시군       경기도 화성시  378418   0.152      화성시
 
 [8039 rows x 5 columns],
 None)

In [8]:
pop.set_index('시군구', inplace=True)
pop

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,61969,0.056
고양시덕양구,구,경기도 고양시 덕양구,495181,0.107
고양시일산동구,구,경기도 고양시 일산동구,293752,0.108
고양시일산서구,구,경기도 고양시 일산서구,288976,0.100
과천시,시군,경기도 과천시,78654,0.110
...,...,...,...,...
포천시,시군,경기도 포천시,158413,0.118
하남시,시군,경기도 하남시,137291,0.140
하남시,시군,경기도 하남시,137291,0.140


In [9]:
map = folium.Map([37.4, 127.1], zoom_start=9, tiles='stamen toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,                       # 단계구분도로 보여줄 데이터
    columns=[pop.index, pop.인구소멸비율],        # DF에서 추출할 컬럼
    fill_color='Pastel2',                      # colormap
    key_on='properties.name'
).add_to(map)

map